# Accessing SDSS-V Astra Products (APOGEE + BOSS)
This notebook demonstrates methods that can be used for programmatic access to SDSS-V data products produced by [Astra](https://sdss-astra.readthedocs.io/en/latest/index.html), the primary analysis framework for both APOGEE and BOSS spectra obtained as part of the Milky Way Mapper (MWM) surveys. Prior to running this notebook, you should have already done the following:

* If not yet an official SDSS-V collaboration member:
  * Sign up for an SDSS-V Twiki Account (follow [instructions here](https://wiki.sdss.org/))
  * Follow the [SDSS-V Welcome instructions](https://sdss-wiki.atlassian.net/wiki/spaces/SDSS/pages/13343105/Welcome+to+SDSS-V) for new collaborators
* Get the SDSS-V data access credentials so you can access [https://data.sdss5.org/sas/sdsswork/](https://data.sdss5.org/sas/sdsswork/)
* [Create a .netrc file](https://sdss-access.readthedocs.io/en/latest/auth.html) in your home directory with SDSS-V username and password

In [ ]:
import os
import io
import sys
import time
import requests
import subprocess
import numpy as np
import pandas as pd
import astropy.units as u
import matplotlib.pyplot as plt

from netrc import netrc
from astropy.io import fits
from astropy.coordinates import SkyCoord
from astropy.coordinates import match_coordinates_sky
from astropy.visualization import PercentileInterval
from astroquery.vizier import Vizier
from IPython.display import display, Markdown
from matplotlib.gridspec import GridSpec

# SDSS-V Tools
sys.path.insert(1, '../')
import sdssv_utils as sdssv

# APOGEE Spectra
The [APOGEE instruments](https://www.sdss.org/instruments/apogee-spectrographs/) are near-IR spectrographs covering the H-band (1.50 - 1.71 $\mu$m) at an approximate resolution of $R\sim22,500$. There are two nearly-identical APOGEE spectrographs, one located in the northern hemisphere at Apache Point Observatory (APO), and one in the southern hemisphere at Las Campanas Observatory (LCO), both on 2.5-m telescopes. Each spectrograph has 300 fibers. The northern APO spectrograph has 2-arcsec fibers while the souther LCO spectrograph has 1.3-arcsec fibers, both with robotic fiber positioners.

# BOSS Spectra
The [BOSS instruments](https://www.sdss.org/instruments/boss-spectrographs/) are optical spectrographs with wavelength coverage of 3600-10400 Angstrom and resolutions of R~2000. There are two identical BOSS spectrographs, one located in the northern hemisphere at Apache Point Observatory (APO), and one in the southern hemisphere at Las Campanas Observatory (LCO), both on 2.5-m telescopes. Each spectrograph has 500, 2-arcsec fibers with robotic fiber positioners.

# Astra Data Products

Astra uses and produces a large variety of data products. This notebook won't cover everything, but will touch on a few of perhaps the more commonly used products that Astra offers. If you'd like to know more about Astra, check out the documentation website [HERE](https://sdss-astra.readthedocs.io/en/latest/user/datamodels.html), or if you have an SDSS-V Twiki acccount already, you can see more up-to-date info [HERE](https://sdss-wiki.atlassian.net/wiki/spaces/MWM/pages/14660061/Astra). Here is a list of some data products this notebook will examine:

* **mwmAllStar**: A summary file listing all objects that have been processed by the Astra framework. This file can be downloaded and cross-matched against source lists and then used to construct URLs for downloading mwmVisit or mwmStar files that contain the BOSS/APOGEE observed spectra. This file also contains metadata such as S/N and RV related to the analysis of the stacked spectra.
* **mwmVisit**: These files contain the individual spectra for a given source. Filenames have the structure `mwmVisit-<ASTRA_VERSION>-<SDSS_ID>.fits`, where both the Astra version should be sdeterminedet by the choice of Internal Product Launch (IPL 1, 2, or 3), and the SDSS-ID (e.g. 88912327) can be obtained from the mwmAllStar or mwmAllVisit files. All wavelengths are velocity shifted to the source rest frame, and each individual spectrum is resampled onto a common wavelength grid.
* **mwmStar**: These files contained the summed BOSS/APOGEE spectra for a given source. Filenames have the structure `mwmStar-<ASTRA_VERSION>-<SDSS_ID>.fits`. Wavelengths are again velocity shifted to the source rest frame.

## Set Some Global Variables
Supported Options:
* `VERSION`:  <span style="font-family:Courier New">ipl-1, ipl-2, ipl-3</span>

In [ ]:
IPL_VERSION = "ipl-3" 
ASTRA_VERSION = sdssv.get_IPL_ASTRA_pipeline_version(IPL_VERSION)
MWM_DIR = f"./mwmall/{IPL_VERSION}"
SAS_BASE = f"https://data.sdss5.org/sas/{IPL_VERSION}"
REMOTE_DIR = f"{SAS_BASE}/spectro/astra/{ASTRA_VERSION}"
NETFILE = f"{os.path.expanduser('~')}/.netrc"
MWMALL_DIR = "./mwmall"  # Local directory where mwmAll files exists

# Download the mwmAll File

In [ ]:
# Make sure MWMALL_DIR exists
if not os.path.exists(MWMALL_DIR):
    os.makedirs(MWMALL_DIR)

# Set filenames
mwmall_local_filename = f"mwmAllStar-{ASTRA_VERSION}.fits"               # Local decompressed file
mwmall_SAS_filename = f"{mwmall_local_filename}.gz"                      # Remote file
mwmall_local_filepath = f"{MWMALL_DIR}/mwmAllStar-{ASTRA_VERSION}.fits"  # Local decompressed file full path
mwmall_SAS_filepath = f"{MWMALL_DIR}/{mwmall_local_filename}.gz"         # Remote file full path
mwmall_url = f"{REMOTE_DIR}/summary/{mwmall_SAS_filename}"               # URL to remote file

# Check if decompressed file already exists
if not os.path.isfile(f"{MWMALL_DIR}/{mwmall_local_filename}"):

    # Download the file
    sdssv.load_file(
        mwmall_url, 
        f"{MWMALL_DIR}/{mwmall_SAS_filename}", 
        NETFILE, 
        SAS_BASE, 
        pbar=True, 
        overwrite=False
    )

    # Decompress the file
    if os.path.isfile(f"{MWMALL_DIR}/{mwmall_SAS_filename}"):
        print('Decompressing mwmAll file...',end='',flush=True)
        time.sleep(1)
        subprocess.run(['gunzip','-f',f"{MWMALL_DIR}/{mwmall_SAS_filename}"])
        print('Finised')
else:
    print(f'Existing mwmAll file found at {mwmall_local_filename}')

# Load mwmAll file
with fits.open(f"{MWMALL_DIR}/{mwmall_local_filename}") as hdu:
    mwmall = hdu[1].data

# Create mwmAll coordinates object
mwmall_ra = mwmall.ra
mwmall_dec = mwmall.dec
mwmall_coord = SkyCoord(
    ra=mwmall_ra*u.deg,
    dec=mwmall_dec*u.deg,
    frame='icrs'
)

# Cross-Match Objects from a Specific SDSS-V Carton with mwmAll File
Using Cartin ID = 632 which is the compact binary cartons (mwm_cb_uvex1)

In [ ]:
# Query carton metadata
carton_result = sdssv.sdssv_carton_query(632, qlimit=500000)

# Get source coordinates
sample_ra = carton_result.ra.values
sample_dec = carton_result.dec.values

# Perform cross match
radius = 1.0 # arcsec
idx_sample, idx_mwmall = sdssv.multiObject_cross_match(
    sample_ra, 
    sample_dec,
    mwmall_coord,
    radius=radius
)

# Create dataframes with objects matched to spectra
mwmall_sample_match = pd.DataFrame(
    mwmall[idx_mwmall].tolist(),
    columns=mwmall.columns.names
)

# Get objects with both BOSS and APOGEE Spectra
mwmall_sample_match = mwmall_sample_match[
    (mwmall_sample_match.n_apogee_visits > 1) &
    (mwmall_sample_match.n_boss_visits > 1)
].reset_index(drop=True)

# Print out crossmatch results
Nsample_unique = len(mwmall_sample_match.sdss_id.unique())
print(
    f"\n{Nsample_unique} Unique Sources " +
    f"Matched to {len(mwmall_sample_match)} Astra mwmStar Files\n"
)

# Download mwmStar and mwmVisit Files for Single Object

In [ ]:
# Select an object
objID = 45 # 12 is CV
sdssID = str(mwmall_sample_match.sdss_id[objID])
id1 = sdssID[-4:-2]
id2 = sdssID[-2:]

# Generate mwmStar Filename
mwmstar_filename = f"mwmStar-{ASTRA_VERSION}-{sdssID}.fits"
mwmstar_url = f"{REMOTE_DIR}/spectra/star/{id1}/{id2}/{mwmstar_filename}"

# Generate mwmVisit Filename
mwmvisit_filename = f"mwmVisit-{ASTRA_VERSION}-{sdssID}.fits"
mwmvisit_url = f"{REMOTE_DIR}/spectra/visit/{id1}/{id2}/{mwmvisit_filename}"

_ = sdssv.load_file(mwmstar_url, mwmstar_filename, NETFILE, SAS_BASE, pbar=True, overwrite=False)
_ = sdssv.load_file(mwmvisit_url, mwmvisit_filename, NETFILE, SAS_BASE, pbar=True, overwrite=False)

# Load mwmStar and mwmVisit Files

In [ ]:
# Load mwmStar file
with fits.open(mwmstar_filename) as hdul:
    # There are also LCO extensions, but only APO is provided in current IPLs
    stardat_boss_apo = hdul['BOSS/APO'].data
    stardat_apogee_apo = hdul['APOGEE/APO'].data

    # Get some metadata
    Nboss_star_apo = stardat_boss_apo.n_good_visits
    Napogee_star_apo = stardat_apogee_apo.n_good_visits

# Load mwmStar file
with fits.open(mwmvisit_filename) as hdul:
    # There are also LCO extensions, but only APO is provided in current IPLs
    visitdat_boss_apo = hdul['BOSS/APO'].data
    visitdat_apogee_apo = hdul['APOGEE/APO'].data

    # Get some metadata
    Nboss_visit_apo = len(visitdat_boss_apo)
    Napogee_visit_apo = len(visitdat_apogee_apo)

# Plot the Spectra

In [ ]:
# Plot the spectra
fig = plt.figure(figsize=(12,10))
gs = GridSpec(2,1)
ax = fig.add_subplot(gs[0])
bx = fig.add_subplot(gs[1])


""" Plot BOSS Spectra """

if len(stardat_boss_apo) > 0:
    boss_good_idx = (stardat_boss_apo.ivar[0] > 0)
    ax.plot(
        stardat_boss_apo.wavelength[0][boss_good_idx], 
        stardat_boss_apo.flux[0][boss_good_idx],
        c='cornflowerblue', lw=1, label=f'BOSS COADD (Nexp = {Nboss_star_apo[0]})'
    );
if Nboss_visit_apo > 0:
    all_boss_flux = []
    for i in range(Nboss_visit_apo):
        if i == 0:
            label = f'BOSS Visits (Nexp = {Nboss_visit_apo})'
        else:
            label = '_none'
        boss_good_idx = (visitdat_boss_apo.ivar[0] > 0)
        all_boss_flux += list(visitdat_boss_apo.flux[i][boss_good_idx])
        bx.plot(
            stardat_boss_apo.wavelength[0][boss_good_idx], 
            visitdat_boss_apo.flux[i][boss_good_idx],
            c='cornflowerblue', lw=1, alpha=0.35, label=label
        );


""" Plot APOGEE Spectra """

if len(stardat_apogee_apo):
    apogee_good_idx = (stardat_apogee_apo.ivar[0] > 0)
    ax.plot(
        stardat_apogee_apo.wavelength[0][apogee_good_idx], 
        stardat_apogee_apo.flux[0][apogee_good_idx],
        c='indianred', lw=1, label=f'APOGEE COADD (Nexp = {Napogee_star_apo[0]})'
    );
if Napogee_visit_apo > 0:
    for i in range(Napogee_visit_apo):
        if i == 0:
            label = f'APOGEE Visits (Nexp = {Napogee_visit_apo})'
        else:
            label = '_none'
        apogee_good_idx = (visitdat_apogee_apo.ivar[0] > 0)
        bx.plot(
            stardat_apogee_apo.wavelength[0][apogee_good_idx], 
            visitdat_apogee_apo.flux[i][apogee_good_idx],
            c='indianred', lw=1, alpha=0.35, label=label
        );
ax.legend(loc='upper right')
bx.legend(loc='upper right')

# Set XY limits
PI = PercentileInterval(99.)
flux_range = PI.get_limits(sorted(all_boss_flux)[100:-100])
y_lowlim = flux_range[0]
y_upplim = flux_range[1]
yrange = y_upplim -  y_lowlim
ax.set_ylim(0.0-0.1*yrange, y_upplim+0.5*yrange)
bx.set_ylim(ax.get_ylim()[0], 1.0*ax.get_ylim()[1])

# Add axis labels
ax.set_xlabel('Wavelength ($\mathrm{\AA}$)',fontsize=14)
bx.set_xlabel('Wavelength ($\mathrm{\AA}$)',fontsize=14)
ax.set_ylabel('$f_{\lambda}$   ($10^{-17}$ $\mathrm{erg/s/cm^2/\AA}$)',fontsize=14)
bx.set_ylabel('$f_{\lambda}$   ($10^{-17}$ $\mathrm{erg/s/cm^2/\AA}$)',fontsize=14)

# Add title
obj_gmag = mwmall_sample_match.g_mag[objID]
obj_RV = mwmall_sample_match.v_rad[objID]
obj_ra = mwmall_sample_match.ra[objID]
obj_dec = mwmall_sample_match.dec[objID]
obj_coord = SkyCoord(
    ra=obj_ra*u.deg, dec=obj_dec*u.deg, frame='icrs'
)
obj_radec = obj_coord.to_string('hmsdms',sep=':',precision=2)
title = f"SDSS-ID = {sdssID} (G-mag = {obj_gmag:.2f}), RA-Dec = {obj_radec},  RV = {obj_RV:.2f} km/s"
ax.set_title(title,fontsize=14)

# Set grid and tick params
ax.grid(ls=":", c='silver')
bx.grid(ls=":", c='silver')
ax.set_axisbelow(True)
bx.set_axisbelow(True)
ax.minorticks_on()
bx.minorticks_on()
ax.tick_params(which='both',top=True,right=True,direction='in',labelsize=13)
bx.tick_params(which='both',top=True,right=True,direction='in',labelsize=13)
